In [23]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import os

# Input FASTA file
sequence_file = "T.thermophila.genome.fasta"
gff_file = "Modified Appendix 1 (sRNA annotations GFF format)_cleaned up_SRL.csv"
output_fasta_file = "outputTetTherm50dif.fasta"

# Load the sequence file
sequences = SeqIO.to_dict(SeqIO.parse(sequence_file, "fasta"))

# Create a list to store SeqRecord objects for the extracted sequences
extracted_sequences = []

# Read the GFF data from the CSV file using pandas
gff_data = pd.read_csv(gff_file)

# Mapping between CSV chromosome names and FASTA chromosome names
chromosome_mapping = {
    'chr_085': 'MacChr85_MicChr3',
    'chr_082': 'MacChr82_MicChr3',
    'chr_128': 'MacChr128_MicChr4',
    'chr_050': 'MacChr50_MicChr1',
    'chr_124': 'MacChr124_MicChr4',
    'chr_009': 'MacChr9_MicChr1',
    'chr_010': 'MacChr10_MicChr1',
    'chr_006': 'MacChr6_MicChr1',
    'chr_175': 'MacChr175_MicChr5',
    'chr_075': 'MacChr75_MicChr3',
    'chr_131': 'MacChr131_MicChr4',
    'chr_027': 'MacChr27_MicChr1',
    'chr_106': 'MacChr106_MicChr3',
    'chr_114': 'MacChr114_MicChr4',
    'chr_067': 'MacChr67_MicChr2',
    'chr_097': 'MacChr97_MicChr3',
    'chr_008': 'MacChr8_MicChr1',
    'chr_019': 'MacChr19_MicChr1',
    'chr_169': 'MacChr169_MicChr5',
    'chr_107': 'MacChr107_MicChr3',
    'chr_066': 'MacChr66_MicChr2',
    'chr_112': 'MacChr112_MicChr4',
    'chr_119': 'MacChr119_MicChr4',
    'chr_025': 'MacChr25_MicChr1',
    'chr_052': 'MacChr52_MicChr1',
    'chr_116': 'MacChr116_MicChr4',
    'chr_178': 'MacChr178_MicChr5',
    'chr_026': 'MacChr26_MicChr1',
    'chr_102': 'MacChr102_MicChr3',
    'chr_179': 'MacChr179_MicChr5',
    'chr_134': 'MacChr134_MicChr4',
    # Add more mappings as needed
}

# Print unique chromosome names from both files
print("Chromosomes in CSV:", gff_data['Chromosome Location'].unique())
print("Chromosomes in FASTA:", sequences.keys())

# Iterate through each row in the GFF data
for index, row in gff_data.iterrows():
    start = row['Starting Nucleotide']
    end = row['Ending Nucleotide']
    csv_chromosome = row['Chromosome Location']
    IDColumn = gff_data.columns.get_loc("ID=")
    sRNA_ID = row.iloc[IDColumn + 1].split(";")[0].replace("ID=", "")

    # Map the CSV chromosome name to the corresponding FASTA chromosome name
    fasta_chromosome = chromosome_mapping.get(csv_chromosome)

    if start > end:
        start, end = end, start

    if fasta_chromosome:
        difference = 50
        start = max(1, start - difference)
        end = min(len(sequences[fasta_chromosome]), end + difference)

        # Extract the sequence using start and end coordinates
        feature_sequence = sequences[fasta_chromosome].seq[start: end]

        description = f"Index:{index} Extracted from {sequence_file}, Chromosome: {fasta_chromosome},  Coordinates: {start}-{end}"

        # Create a SeqRecord object for the extracted sequence
        extracted_record = SeqRecord(
            feature_sequence,
            id=f"{sRNA_ID}",
            description=description,
        )

        extracted_sequences.append(extracted_record)

# Write the extracted sequences to a new FASTA file in the current working directory
SeqIO.write(extracted_sequences, output_fasta_file, "fasta")

print(f"Extracted sequences saved to {output_fasta_file}")


Chromosomes in CSV: ['chr_085' 'chr_082' 'chr_128' 'chr_050' 'chr_124' 'chr_009' 'chr_010'
 'chr_006' 'chr_175' 'chr_075' 'chr_131' 'chr_027' 'chr_106' 'chr_114'
 'chr_067' 'chr_097' 'chr_008' 'chr_019' 'chr_169' 'chr_107' 'chr_066'
 'chr_112' 'chr_119' 'chr_025' 'chr_052          ' 'chr_116' 'chr_178'
 'chr_026' 'chr_102' 'chr_179' 'chr_134']
Chromosomes in FASTA: dict_keys(['MacChr1_MicChr1', 'MacChr2_MicChr1', 'MacChr3_MicChr1', 'MacChr4_MicChr1', 'MacChr5_MicChr1', 'MacChr6_MicChr1', 'MacChr7_MicChr1', 'MacChr8_MicChr1', 'MacChr9_MicChr1', 'MacChr10_MicChr1', 'MacChr11_MicChr1_rDNA', 'MacChr12_MicChr1', 'MacChr13_MicChr1', 'MacChr14_MicChr1', 'MacChr15_MicChr1', 'MacChr16_MicChr1', 'MacChr17_MicChr1', 'MacChr18_MicChr1', 'MacChr19_MicChr1', 'MacChr20_MicChr1', 'MacChr21_MicChr1', 'MacChr22_MicChr1', 'MacChr23_MicChr1', 'MacChr24_MicChr1', 'MacChr25_MicChr1', 'MacChr26_MicChr1', 'MacChr27_MicChr1', 'MacChr28_MicChr1', 'MacChr29_MicChr1', 'MacChr30_MicChr1', 'MacChr31_MicChr1', 'MacC

In [24]:
import os
from Bio import SeqIO

def sanitize_filename(filename):
    # Replace special characters with underscores
    return filename.replace(":", "_").replace("/", "_").replace("-", "_")

# Input FASTA file
input_fasta_file = "outputTetTherm50dif.fasta"

# Output folder
output_folder = "OutputFolderSingleSeq"
os.makedirs(output_folder, exist_ok=True)
# Read the input file and grab all sequences
with open(input_fasta_file, "r") as input_handle:
    records = SeqIO.to_dict(SeqIO.parse(input_handle, "fasta"))

    # Write each non-empty sequence to an individual file in the output folder
    for seq_id, sequence in records.items():
        # Skip empty sequences
        if not str(sequence.seq).strip():
            continue

        # Sanitize the filename
        sanitized_seq_id = sanitize_filename(seq_id)

        output_file = os.path.join(output_folder, f"{sanitized_seq_id}.fasta")
        with open(output_file, "w") as output_handle:
            SeqIO.write(sequence, output_handle, "fasta")

print(f"All non-empty sequences have been grabbed and placed into {output_folder}.")


All non-empty sequences have been grabbed and placed into OutputFolderSingleSeq.


In [28]:
import os
from Bio import SeqIO
import subprocess

#sequence_file = "T.thermophila.genome.fasta"
#sequence_file = "T.elliotti.genome.fasta"
#sequence_file = "T.borealis.genome.fasta"
sequence_file = "T.malaccensis.genome.fasta"
#change ouput file name to match sequence_file
output_folder = "TetOutputBlastn"
# Define BLAST commands
makeDatabase_command = f"makeblastdb -in {sequence_file} -dbtype nucl -out target_database"
# Run makeblastdb to create the database
subprocess.run(makeDatabase_command, shell=True)

# Specify the folder containing your sequence files
sequence_folder = "OutputFolderSingleSeq"

# List all files in the folder
sequence_files = [f for f in os.listdir(sequence_folder) if f.endswith(".fasta")]

# Create a dictionary to store the top-scoring sequence for each query
top_scoring_sequences = {}

# E-value threshold
evalue_threshold = 1e-5

# Iterate over each sequence file
for sequence_file in sequence_files:
    # Read the sequence file using Biopython
    sequences = SeqIO.to_dict(SeqIO.parse(os.path.join(sequence_folder, sequence_file), "fasta"))

    # Perform BLAST for each query sequence
    for header, sequence in sequences.items():
        # Define the BLASTn command with the current query sequence
        blastn_command = [
            "blastn",
            "-query", "-",
            "-db", "target_database",
            "-outfmt", "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore"
        ]

        # Run BLAST with the sequence as input
        blast_process = subprocess.Popen(blastn_command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, universal_newlines=True)
        blast_output, _ = blast_process.communicate(input=f">{header}\n{sequence.seq}")

        # Process results and save the top-scoring sequence if e-value is below the threshold
        best_result = None
        for line in blast_output.strip().split('\n'):
            fields = line.strip().split('\t')

            # Check if there are enough fields and the e-value is present
            if len(fields) > 10:
                evalue = float(fields[10])

                # Check if it's the top-scoring sequence and meets the e-value threshold
                if best_result is None or (evalue < best_result['evalue'] and evalue < evalue_threshold):
                    best_result = {
                        'header': f"{header} from {sequence_file}",  # Include sequence file information in the header
                        'sequence': sequence.seq,
                        'evalue': evalue
                    }

        # Save the top-scoring sequence to the dictionary if it meets the threshold
        if best_result is not None and best_result['evalue'] < evalue_threshold:
            top_scoring_sequences[header] = best_result

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save all the top-scoring sequences to a single FASTA file
output_file_name = os.path.join(output_folder, "TetMal_top_scoring_sequences.fasta")
#change Fasta file name to match sequence_file
with open(output_file_name, "w") as output_file:
    for header, result in top_scoring_sequences.items():
        output_file.write(f">{result['header']} (Top e-value: {result['evalue']})\n")
        output_file.write(f"{result['sequence']}\n")
